In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.95'
sys.path.append('/mnt/local/data/vtrifonov/prec-learning-Notay-loss/')

In [5]:
from copy import deepcopy

from jax import random

from train import make_neural_prec_model
from utils import params_count

In [9]:
default_precorrector_mlp_config = {
    'layer_type': 'Conv1d',
    'static_diag': False,
    'alpha': 0.,
    'mlp': {
        'features': [1, 16, 1],
        'N_layers': 2,
    }
}

default_precorrector_gnn_config = {
    'layer_type': 'Conv1d',
    'use_nodes': False,
    'node_upd_mlp': False,
    'static_diag': True,
    'alpha': 0.,
    'edge_enc': {
        'features': [1, 16, 16],
        'N_layers': 2,
    },
    'edge_dec': {
        'features': [16, 16, 1],
        'N_layers': 2,
    },
    'mp': {
        'edge_upd': {
            'features': [18, 16, 16],
            'N_layers': 2,
        },
        'node_upd': {
            'features': [17, 1, 1],
            'N_layers': 2,
        },
        'mp_rounds': 5,
        'aggregate_edges': 'max'
    }
}

default_naive_gnn_config = {
    'layer_type': 'ConstantConv1d',
    'node_enc': {
        'features': [1, 16, 16],
        'N_layers': 2,
    },
    'edge_enc': {
        'features': [1, 16, 16],
        'N_layers': 2,
    },
    'edge_dec': {
        'features': [16, 16, 1],
        'N_layers': 2,
    },
    'mp': {
        'edge_upd': {
            'features': [48, 16, 16],
            'N_layers': 2,
        },
        'node_upd': {
            'features': [32, 16, 16],
            'N_layers': 2,
        },
        'mp_rounds': 5,
        'aggregate_edges': 'sum'
    }
}

# model_type in {'precorrector_gnn', 'precorrector_gnn_multiblock', 'naive_gnn', 'precorrector_mlp'}

# PreCorrector paper

In [4]:
params_count(make_neural_prec_model(
    key = random.PRNGKey(42),
    config = default_precorrector_gnn_config,
    model_type = 'precorrector_gnn'
))

2025-01-30 22:48:20.471344: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


1170

# NaiveGNN and PreCorrector with nodes update MLP and with nodes encoder

In [10]:
params_count(make_neural_prec_model(
    key = random.PRNGKey(42),
    config = default_naive_gnn_config,
    model_type = 'naive_gnn'
))

2753

# PreCorrector with multi block message-passing protocol

In [11]:
params_count(make_neural_prec_model(
    key = random.PRNGKey(42),
    config = default_precorrector_gnn_config,
    model_type = 'precorrector_gnn_multiblock'
))

3474

# PreCorrector with MLP instead of GNN

In [12]:
params_count(make_neural_prec_model(
    key = random.PRNGKey(42),
    config = default_precorrector_mlp_config,
    model_type = 'precorrector_mlp'
))

50